In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import pickle
df = pd.read_csv("creditcard.csv")
df_small = df.sample(10000, random_state=42)
X = df_small.drop("Class", axis=1)
y = df_small["Class"]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)
model = RandomForestClassifier(n_estimators=20, random_state=42)
model.fit(X_train, y_train)
pickle.dump(model, open("fraud_model.pkl", "wb"))
pickle.dump(scaler, open("scaler.pkl", "wb"))
print("Fast Fraud Detection Model trained & saved!")


Fast Fraud Detection Model trained & saved!


In [18]:
from flask import Flask, request, jsonify
import pickle
import pandas as pd
import numpy as np
model = pickle.load(open("fraud_model.pkl", "rb"))
scaler = pickle.load(open("scaler.pkl", "rb"))
app = Flask(__name__)
@app.route("/")
def home():
    return "Fraud Detection API is Running!"
@app.route("/predict", methods=["POST"])
def predict():
    data = request.get_json()
    input_features = pd.DataFrame([data["features"]], columns=X.columns)
    scaled = scaler.transform(input_features)
    prediction = model.predict(scaled)[0]
    return jsonify({"fraud_prediction": int(prediction)})


In [19]:
# Use Flask's test client to test API inside Colab
client = app.test_client()
sample = X.iloc[0].tolist()
response = client.post("/predict", json={"features": sample})
print("API Output:", response.get_json())


API Output: {'fraud_prediction': 1}
